In [248]:
import torch
import topmost  
torch.cuda.is_available()

import topmost
from topmost.data import download_dataset

device = "cuda" # or "cpu"
dataset_dir = "./datasets/20NG"
download_dataset('20NG', cache_path='./datasets')

dataset = topmost.BasicDataset(dataset_dir)

100%|██████████| 11.9M/11.9M [00:00<00:00, 18.7MB/s]


train_size:  11314
test_size:  7532
vocab_size:  5000
average length: 110.543


In [ ]:
import topmost.utils
import topmost.utils._utils
from topmost import eva


class Beta_env():
    def __init__(self, dataset, max_steps, num_topics, embed_size, device, reference_corpus, num_top_words = 5, random = False):
        self.device = device
        self.num_topics = num_topics
        self.embed_size = embed_size
        self.random_init = random
        self.step_counter = 0
        self.max_steps = max_steps
        self.num_top_words = num_top_words
        self.reference_corpus = reference_corpus
        self.dataset = dataset
        self.vocab = dataset.vocab
        self.vocab_size = len(self.vocab)

        self.str_to_embeds = {
            dataset.vocab[i]:torch.tensor(
                dataset.pretrained_WE[i], device = device
            ) for i in range(0, len(dataset.vocab))
        }
        
        self.beta, self.topic_embeds = self.get_starting_state()


    def get_starting_state(self):
        if self.random_init:
            beta = torch.rand(size=(self.num_topics, self.vocab_size), device = self.device)
            topic_embeds = torch.rand(size = (self.num_topics, self.embed_size), device = self.device)
        else:
            beta, topic_embeds = None, None
        return beta, topic_embeds

    def reset(self):
        self.step_counter = 0
        self.beta, self.topic_embeds = self.get_starting_state()
        return torch.concatenate((self.beta, self.topic_embeds), dim = 1)
    
    
    def compute_cosine_similarity(self, list_txt):
        result = torch.zeros((1,1), device = self.device)
        for i in range(self.num_topics):
            for word in list_txt[i].split():
                we = self.str_to_embeds[word]
                result += torch.cosine_similarity(we, self.topic_embeds[i, :], dim = 0)
        return result.cpu().numpy().sum()
        
    def calculate_reward(self):
        top_words = self.get_top_words(False)
        total_TD =  eva.topic_diversity._diversity(top_words)

        total_cosine_similarity = self.compute_cosine_similarity(top_words)
        
        return  (total_TD / self.num_topics + total_cosine_similarity / (self.num_topics * self.num_top_words)) / 2
        

    def get_top_words(self, verbose = True):
        top_word_list = topmost.utils._utils.get_top_words(self.beta.cpu().numpy(), self.vocab, num_top_words=self.num_top_words, verbose=verbose)
        return top_word_list

    def step(self, action):
        self.beta += action
        new_state = torch.concatenate((self.beta, self.topic_embeds), dim = 1)
        reward = self.calculate_reward()
        self.step_counter += 1
        done = False
        if self.step_counter == self.max_steps:
            done = True
            self.step_counter = 0
        
        self.state = new_state
        return self.state, reward, done

In [313]:
env = Beta_env(
    dataset = dataset,
    max_steps=3,
    num_topics=5,
    embed_size=200,
    device = "cuda",
    reference_corpus=dataset.train_texts,
    random = True
)
# x = env.get_top_words()
env.calculate_reward()

0.11632000327110291

In [323]:
action = torch.randn(size = (5, 5000), device = device)
env.step(action)

(tensor([[ 4.7914,  3.7741, -2.6307,  ...,  0.7557,  0.8145,  0.8498],
         [-3.6298, -1.4636,  2.3268,  ...,  0.9754,  0.9951,  0.3757],
         [-1.7730,  4.7649,  2.6474,  ...,  0.8158,  0.5052,  0.5368],
         [ 0.4783,  0.1749, -1.8445,  ...,  0.9176,  0.6082,  0.7910],
         [-0.1448,  1.8021,  0.7093,  ...,  0.0653,  0.1500,  0.3311]],
        device='cuda:0'),
 0.11993046402931214,
 False)